
# A Notebook to explore gridded climate time series data and DHSVM modeling of the Sauk-Suiattle Watershed  
<img src= "http://www.sauk-suiattle.com/images/Elliott.jpg"
style="float:left;width:200px;padding:20px">   
This data is compiled to digitally observe the Sauk-Suiattle Watershed, powered by HydroShare. <br />
<br />
*Use this Jupyter Notebook to:* <br /> 
Download and generate lists of gridded climate points for a watershed<br />
Download Livneh daily 1/16 degree gridded climate data, <br /> 
Download WRF daily 1/16 degree gridded climate data, <br /> 
Visualize daily, monthly, and annual temperature and precipitation data. <br /> 
Calculate Long-term Mean Monthly Bias Corrections for WRF using Livneh Low Elevation data<br /> 
Bias correct each Livneh grid cell using bias corrected WRF (use to correct Livneh 2013 and MACA data). <br />
Visualize daily, monthly, and annual temperature and precipitation data with corrected results. <br /> 
Update VIC model soil input (optional). <br /> 
Save results back to HydroShare. <br /> 
 <br /> <br /> <img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:right;width:120px;padding:20px">  
#### A Watershed Dynamics Model by the Watershed Dynamics Research Group in the Civil and Environmental Engineering Department at the University of Washington 

This is the Livneh bias correction to WRF, that is bias corrected to the Livneh low elevation long term mean, which has been spatially averaged (with a few experimental GIS selections).  

## 1.  HydroShare Setup and Preparation

To run this notebook, we must import several libaries. These are listed in order of 1) Python standard libraries, 2) hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading and creation, and 3) the observatory_gridded_hydromet library that is downloaded with this notebook. 

In [1]:
#Python libraries available on CUAHSI JupyterHub 
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
%matplotlib inline
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')
#HydroShare Utilities
from utilities import hydroshare
#UW Watershed Dynamics Lab Utilities
#from utilities import observatory_gridded_hydromet as ogh

Establish a secure connection with HydroShare by instantiating the hydroshare class that is defined within hs_utils. In addition to connecting with HydroShare, this command also sets and prints environment variables for several parameters that will be useful for saving work back to HydroShare. 

In [2]:
hs=hydroshare.hydroshare()
#homedir = ogh.mapContentFolder(str(os.environ["HS_RES_ID"]))
#print('Data will be loaded from and save to:'+homedir)

Adding the following system variables:
   HS_USR_NAME = ChristinaBandaragoda
   HS_RES_ID = f0f90f5645864e0d9c0e0209d0095d74
   HS_RES_TYPE = genericresource
   JUPYTER_HUB_IP = jupyter.cuahsi.org

These can be accessed using the following command: 
   os.environ[key]

   (e.g.)
   os.environ["HS_USR_NAME"]  => ChristinaBandaragoda
Successfully established a connection with HydroShare


If you are curious about where the data is being downloaded, click on the Jupyter Notebook dashboard icon to return to the File System view.  The homedir directory location printed above is where you can find the data and contents you will download to a HydroShare JupyterHub server.  At the end of this work session, you can migrate this data to the HydroShare iRods server as a Generic Resource. 

In [3]:
mappingfile='/home/jovyan/work/notebooks/data/f0f90f5645864e0d9c0e0209d0095d74/f0f90f5645864e0d9c0e0209d0095d74/data/contents/monkeysonatree.csv'
statefileexample='/home/jovyan/work/notebooks/data/MetSim-develop/tests/data/state.nc'

from utilities import tonic
import xarray

In [11]:
map_df= pd.read_csv(mappingfile)
#print(map_df)

data = tonic.calc_grid(map_df.LAT, map_df.LONG_, decimals=4)
print(data)

Calculating grid size now...
found 14 unique lons
found 11 unique lats
Created a target grid based on the lats and lons in the input file names
Grid Size: (11, 14)
{'lon': (array([-121.7188, -121.6562, -121.5938, -121.5312, -121.4688, -121.4062,
       -121.3438, -121.2812, -121.2188, -121.1562, -121.0938, -121.0312,
       -120.9688, -120.9062]), ('lon',), {'units': 'degrees_east', 'long_name': 'longitude coordinate'}), 'mask': array([[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'lat

In [5]:
dataA=data['mask']
print(dataA)
dataA.shape

[[0 0 0 0 0 1 1 1 1 0 0 0 0 0]
 [0 0 0 1 1 1 1 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 1 1 1 1 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0]]


(11, 14)

In [6]:
data3 = np.expand_dims(dataA, axis=0)
data3.shape

(1, 11, 14)

In [7]:
ds = xarray.Dataset({'mask': (['x', 'y', 'time'],  data3)}                )

In [24]:
ds.to_netcdf('sauk_domain.nc')
!cp 'sauk_domain.nc' /home/jovyan/work/notebooks/data/MetSim-develop/tests/data

In [10]:
print(map_df.LONG_.keys())

RangeIndex(start=0, stop=98, step=1)


In [23]:
temp = 15 + 8 * np.random.randn(11, 14, 3)

precip = 10 * np.random.rand(11, 14, 3)

lon = data['lon']#[[-99.83, -99.32], [-99.79, -99.23]]

lat = data['lat'] #[[42.25, 42.21], [42.63, 42.59]]

# for real use cases, its good practice to supply array attributes such as
# units, but we won't bother here for the sake of brevity
stateds = xarray.Dataset({'mask': (['x', 'y', 'time'],  data3),
                         'time': pd.date_range('1950-01-01', periods=14),
                         'reference_time': pd.Timestamp('1951-12-31')})
stateds.to_netcdf('sauk_state.nc')

In [25]:
# use bash to look at your files
#!ls
!cp sauk_state.nc ../../../../MetSim-develop/tests/data
!cp sauk_domain.nc ../../../../MetSim-develop/tests/data

12186000.streamflow.daily.cms.txt
12189500.streamflow.daily.cms.txt
avg_monthly_temp.png
ChocolateGlacier.streamflow.daily.cms.txt
CoolGlacier.streamflow.daily.cms.txt
data_48.15625_-121.28125
domain.nc
DustyGlacier.streamflow.daily.cms.txt
ELA_all_6_7.txt
ErmineGlacier.streamflow.daily.cms.txt
gl_cov_glac_6_7.txt
HoneycombGlacier.streamflow.daily.cms.txt
monkeysonatree.csv
month_Livneh1915_2011.csv
month_WRF_CIG_1950_2010.csv
observatory_gridded_hydromet_072117.py
observatory_gridded_hydromet_Jimmys082217version.py
observatory_gridded_hydromet.py
Observatory_Sauk_071817.ipynb
Observatory_Sauk_072117.ipynb
Observatory_Sauk_LivBC2WRF_090817.ipynb
Observatory_Sauk_LivBC2WRF_091417.ipynb
Observatory_Sauk_LivBC2WRF_091817.ipynb
Observatory_Sauk_Livbc2WRFbc2LivLlowelevltm.ipynb
Observatory_Sauk_LivLovWRFltm_MetBiasCorrection.ipynb
Observatory_Sauk_LivnehBCtoWRF.ipynb
Observatory_Sauk_MetBiasCorrection.ipynb
Observatory_Sauk_MetHighElevBiasCorrection.ipynb
Observatory_Sauk_MetMidHighElevBias

## 6. Run MetSim to dissagregate daily data to 3-hrly DHSVM inputs

In [ ]:
#get scripts from clone or download on JupyterHub server 
!cp /home/jovyan/work/notebooks/data/MetSim-develop/* . 



import MetSim-develop.metsim



## 8. Save the results back into HydroShare
<a name="creation"></a>

Using the `hs_utils` library, the results of the Geoprocessing steps above can be saved back into HydroShare.  First, define all of the required metadata for resource creation, i.e. *title*, *abstract*, *keywords*, *content files*.  In addition, we must define the type of resource that will be created, in this case *genericresource*.  

***Note:*** Make sure you save the notebook at this point, so that all notebook changes will be saved into the new HydroShare resource.

Move each file on the server within the 'files' list to an :EXISTING" HydroShare Generic Resource content folder.  Parent_resource is the destination resource ID for an existing Generic Resource. Files is a list of filepaths.

In [ ]:
"""
# Talk to CUAHSI about why the add file function is throwing errors
"""
#parent_resource = '0236ae196d204f1cba421787f38dec71'
#files= ['Observatory_Sauk_071117.ipynb'] #os.path.join(homedir,)

#response_json = hs.addContentToExistingResource(resid=parent_resource, content=files)

In [ ]:
!ls

In [ ]:
ThisNotebook='Observatory_Sauk_LivBC2WRF_091417.ipynb' #check name for consistency
liv2013_tar = 'livneh2013.tar.gz'
wrf_tar = 'Salathe2014.tar.gz'
biascorrWRF_liv_tar = 'biascorrWRF_WRFbc.tar.gz'

!tar -zcf {liv2013_tar} livneh2013
!tar -zcf {wrf_tar} Salathe2014
!tar -zcf {biascorrWRF_liv_tar} biascorrWRF_liv

files=[ThisNotebook,
       'observatory_gridded_hydromet.py',
       liv2013_tar,
       wrf_tar,
       biascorrWRF_liv_tar]

In [ ]:
# for each file downloaded onto the server folder, move to a new HydroShare Generic Resource
title = 'Sauk-Suiattle Observatory Bias Correction Results - Livneh 2013 to WRFbc'
abstract = 'This output is a bias correction test to generate a hybrid gridded meteorology product'
keywords = ['Sauk', 'climate', 'WRF','hydrometeorology'] 
rtype = 'genericresource'  

# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title,
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=False)